#### Source Wallet Account

- Hash: tz1SJiGQwuXLyhCLqFosCw2YsCfqqHjTTSGh
- Public Key: edpkuEp219C1UxSuys58uqYCHKFUQD9CyRgnrdr5ADwthGrg2x2L5n
- Secret Key: unencrypted:edsk2wfh9Qp9heaozaS3kV3h4UUBFDdNSFcJJoutz47r9jCLD6hrZZ
- Seed: ostrich snack across neither inflict diamond slogan two toddler junior initial asthma fine keen reform

#### Target Wallet Acount

- Hash: tz1WDXpfSBhTH4knXnQg52s98zEvM1WUDvRG
- Public Key: edpkvBhbnhH44rHDRHze7MnjKmYQZmpQhVjLG2UeYunFWK23bexn7y
- Secret Key: unencrypted:edsk2nzHbTbVHszKyJmXS4tXAfT3vmyR7uWfidjMASivWYp9hJjnw9
- Seed: what spend range blur shadow move runway finish exhibit cage task crisp tumble february sleep

In [293]:
import requests;
import base58check;
import base58;
import pprint
from cryptography.hazmat.primitives.asymmetric.ed25519 import Ed25519PrivateKey
import hashlib
import time

pp = pprint.PrettyPrinter(indent=2)

RPC_ENDPOINT = 'https://rpc.hangzhounet.teztnets.xyz'
TARGET_WALLET = 'tz1WDXpfSBhTH4knXnQg52s98zEvM1WUDvRG'
SOURCE_WALLET = 'tz1SJiGQwuXLyhCLqFosCw2YsCfqqHjTTSGh'

In [294]:
# 1. Prepare data
print('target: ' + requests.get(RPC_ENDPOINT + '/chains/main/blocks/head/context/contracts/' + TARGET_WALLET).json()['balance'])
print('source: ' +  requests.get(RPC_ENDPOINT + '/chains/main/blocks/head/context/contracts/' + SOURCE_WALLET).json()['balance'])

header = requests.get(RPC_ENDPOINT + '/chains/main/blocks/head/header').json()
metadata = requests.get(RPC_ENDPOINT + '/chains/main/blocks/head/metadata').json()
account = requests.get(RPC_ENDPOINT + '/chains/main/blocks/head/context/contracts/' + SOURCE_WALLET).json()
chain_id = requests.get(RPC_ENDPOINT + '/chains/main/chain_id').json()

branch = header['hash']
protocol = metadata['next_protocol']
counter = account['counter']

pp.pprint({
    'hash': branch,
    'protocol': protocol,
    'counter': counter
})

target: 46460741540
source: 88682298577
{ 'counter': '3220646',
  'hash': 'BLNRq4Wq24Lm625JhidTdoTdSEB4FKGvTwstg98e3HBsaY78QaX',
  'protocol': 'PtHangz2aRngywmSRGGvrcTyMbbdpWdpFKuS4uMWxg2RaH9i1qx'}


In [295]:
# 2. Dry Run 
transaction = { 
    'kind': 'transaction',
    'source': SOURCE_WALLET,
    'destination': TARGET_WALLET,
    'fee': '500000',
    'counter': str((int(counter) + 1)),
    'gas_limit': '1040000',
    'storage_limit': '60000',
    'amount': '935385'
}

operation = {
    'branch': branch,
    'contents': [transaction],
    'signature': 'edsigtXomBKi5CTRf5cjATJWSyaRvhfYNHqSUGrn4SdbYRcGwQrUGjzEfQDTuqHhuA8b2d8NarZjz8TRf65WkpQmo423BtomS8Q'
}

response = requests.post(RPC_ENDPOINT + '/chains/main/blocks/head/helpers/scripts/run_operation', json = {
    'operation': operation,
    'chain_id': chain_id
}).json()

In [296]:
# 3. Forge transaction

response = requests.post(RPC_ENDPOINT + '/chains/main/blocks/head/helpers/forge/operations', json = {
    'branch': branch,
    'contents': [transaction]
}).json()

forge_bytes = bytes.fromhex(response)
forged_string = (b'\x03' + forge_bytes).hex()
print('Forged data: ' + forged_string)

Forged data: 03569dbd6388ef80db46b69ce2b10212021c7cb09dbbb743797df8ea9f771f4a8c6c0049219893562ea75cedeefbd6c3aa39fdcdb49075a0c21ea7c9c40180bd3fe0d403d98b39000074074d175f66f74baa66dc6bf10415fcb480e3bc00


In [297]:
# 4. Sign

privatekey_bytes = base58check.b58decode('edsk2wfh9Qp9heaozaS3kV3h4UUBFDdNSFcJJoutz47r9jCLD6hrZZ')[4:][:32]

print(privatekey_bytes.hex())

hash_bytes = bytes.fromhex(hashlib.blake2b(bytes.fromhex(forged_string), digest_size=32).hexdigest())

print(hash_bytes.hex())


privatekey = Ed25519PrivateKey.from_private_bytes(privatekey_bytes)

sign = privatekey.sign(hash_bytes)

print(sign.hex())


edsig =  base58.b58encode_check(b'\x09\xf5\xcd\x86\x12' + sign).decode('utf-8')

print(edsig)


23cc4dc8a63b2039760645926475586480033eff9a830e7938be2b7927330202
61e9456e74973a0b97cfcb860f41944a72a5e9351d782efe724736849cbc0ca2
0e49c4d1a32e55c83359046e713f934342b990e29e394cfd82a6334f7fbd87121d4cfcdfe8dfcfecfa415e6a28b985cc7a41bd0d1ed488e9bf90af986178ef03
edsigtZgBVooT3tDmLAzk52oYfigaHTqaTPgy8d3ycTuDEU4cZyqzvFeQJQpCGLSAFvtP7fhFKA99XDzPZGChzL8cZVXDn2Evwu


In [298]:
# 5. Preapply

preapply_operations = [{
    'protocol': protocol,
    'branch': branch,
    'contents': [transaction],
    'signature': edsig
}]

response = requests.post(RPC_ENDPOINT + '/chains/main/blocks/head/helpers/preapply/operations', json = preapply_operations)

pp.pprint(response.json())


[ { 'contents': [ { 'amount': '935385',
                    'counter': '3220647',
                    'destination': 'tz1WDXpfSBhTH4knXnQg52s98zEvM1WUDvRG',
                    'fee': '500000',
                    'gas_limit': '1040000',
                    'kind': 'transaction',
                    'metadata': { 'balance_updates': [ { 'change': '-500000',
                                                         'contract': 'tz1SJiGQwuXLyhCLqFosCw2YsCfqqHjTTSGh',
                                                         'kind': 'contract',
                                                         'origin': 'block'},
                                                       { 'category': 'fees',
                                                         'change': '500000',
                                                         'cycle': 124,
                                                         'delegate': 'tz1Ke2h7sDdakHJQh8WX4Z372du1KChsksyU',
                                            

In [299]:
# 6. Inject


response = requests.post(RPC_ENDPOINT + '/injection/operation', json = forge_bytes.hex() + sign.hex())

operation_hash = response.json()

pp.pprint(operation_hash)

'ooMb1FFKN6SaEuJRXe6g5aHCHkE5rUaBa6hYkcTFCay4oj348JR'


In [301]:
# 7. Check balances
print('target: ' + requests.get(RPC_ENDPOINT + '/chains/main/blocks/head/context/contracts/' + TARGET_WALLET).json()['balance'])
print('source: ' +  requests.get(RPC_ENDPOINT + '/chains/main/blocks/head/context/contracts/' + SOURCE_WALLET).json()['balance'])

pp.pprint(requests.get('https://api.hangzhou2net.tzkt.io/v1/operations/' + operation_hash).json())





target: 46461676925
source: 88680863192
[ { 'allocationFee': 0,
    'amount': 935385,
    'bakerFee': 500000,
    'block': 'BLR8rAVZxgtDMb6329mFBzrPYybRFPUmQDJzxjgcEvadVG2TQdq',
    'counter': 3220647,
    'gasLimit': 1040000,
    'gasUsed': 1420,
    'hasInternals': False,
    'hash': 'ooMb1FFKN6SaEuJRXe6g5aHCHkE5rUaBa6hYkcTFCay4oj348JR',
    'id': 14907619,
    'level': 509866,
    'sender': {'address': 'tz1SJiGQwuXLyhCLqFosCw2YsCfqqHjTTSGh'},
    'status': 'applied',
    'storageFee': 0,
    'storageLimit': 60000,
    'storageUsed': 0,
    'target': {'address': 'tz1WDXpfSBhTH4knXnQg52s98zEvM1WUDvRG'},
    'timestamp': '2022-02-12T10:46:23Z',
    'type': 'transaction'}]
